In [157]:
from pandas.io.json import json_normalize
from urllib.parse import quote
from urllib.request import Request, urlopen
import requests
import json
import pandas as pd
import datetime as dt
import numpy as np

In [158]:
fldata = pd.read_csv('./data/fldata.csv')
cfdata = pd.read_csv('./data/cfdata.csv')

In [159]:
cfdata

,Unnamed: 0,APVPERMYMD,TRDSTATEGBN,TRDSTATENM,DCBYMD,SITEAREA,BPLCNM,X,Y,region,ADDR,JIGA,at4,po3,num_unv,unv_name,unv_dist
0,0,2013-11-13,3,폐업,2017-10-11,22.53,탭플레이 커피,127.134306,37.525438,강동구,서울특별시 강동구 양재대로87길 11,6350000,6,8,1,한국체육대학교,662
1,1,2015-07-08,3,폐업,2017-07-10,22.00,아카페,127.131877,37.539416,강동구,서울특별시 강동구 진황도로27길 23,4831000,10,10,0,없음,x
2,2,2014-03-20,3,폐업,2017-07-17,74.21,CAFE TENUTO 테누토,127.146158,37.550130,강동구,서울특별시 강동구 구천면로 428,6596000,3,11,0,없음,x
3,3,2014-12-05,3,폐업,2017-06-02,337.19,할리스커피 올림픽공원북문점,127.129210,37.524394,강동구,서울특별시 강동구 강동대로 205,8360000,9,9,1,한국체육대학교,490
4,4,2013-01-03,3,폐업,2017-05-18,30.00,도우드림,127.140339,37.531725,강동구,서울특별시 강동구 풍성로63길 84,4420000,3,10,0,없음,x
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24536,24536,2011-03-02,1,영업/정상,NaN,98.10,카페 엘리,127.032416,37.485785,서초구,서울특별시 서초구 강남대로39길 15-7,0,1,12,2,횃불트리니티 신학대학원대학교,746
24537,24537,2010-09-16,1,영업/정상,NaN,50.00,카페레즈,127.041496,37.481075,서초구,서울특별시 서초구 바우뫼로41길 8,8030000,6,9,3,서울외국어대학원대학교,235
24538,24538,2010-05-27,1,영업/정상,NaN,142.50,마켓 비노플라워,126.996156,37.498874,서초구,서울특별시 서초구 사평대로 106,12190000,5,11,1,가톨릭대학교 성의교정,839
24539,24539,2009-10-29,1,영업/정상,NaN,62.00,까페원더,127.039916,37.484342,서초구,서울특별시 서초구 바우뫼로41길 88,7904000,4,8,3,서울외국어대학원대학교,341


In [160]:
fldata = fldata.drop(['Unnamed: 0'],axis=1)

In [161]:
cfdata = cfdata.drop(['Unnamed: 0'],axis=1)

In [162]:
fldata.iloc[0,2:]

행정동         사직동
계          9831
0~4세        354
5~9세        416
10~14세      363
15~19세      426
20~24세      522
25~29세      649
30~34세      714
35~39세      793
40~44세      842
45~49세      867
50~54세      766
55~59세      771
60~64세      638
65~69세      458
70~74세      410
75~79세      383
80~84세      261
85~89세      113
90~94세       46
95~99세       22
100세 이상      17
Name: 0, dtype: object

In [163]:
date = pd.to_datetime(cfdata['DCBYMD'],format='%Y-%m-%d')
opdate = pd.to_datetime(cfdata['APVPERMYMD'],format='%Y-%m-%d')
cfdata['DCBYMD'] = date
cfdata['APVPERMYMD'] = opdate

cfdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24541 entries, 0 to 24540
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   APVPERMYMD   24541 non-null  datetime64[ns]
 1   TRDSTATEGBN  24541 non-null  int64         
 2   TRDSTATENM   24541 non-null  object        
 3   DCBYMD       8624 non-null   datetime64[ns]
 4   SITEAREA     23486 non-null  object        
 5   BPLCNM       24541 non-null  object        
 6   X            24541 non-null  float64       
 7   Y            24541 non-null  float64       
 8   region       24541 non-null  object        
 9   ADDR         24541 non-null  object        
 10  JIGA         24541 non-null  int64         
 11  at4          24541 non-null  int64         
 12  po3          24541 non-null  int64         
 13  num_unv      24541 non-null  int64         
 14  unv_name     24541 non-null  object        
 15  unv_dist     24541 non-null  object        
dtypes: d

In [8]:
url1 = 'https://dapi.kakao.com/v2/local/search/address.json?query='+ cfdata.ADDR[2]
headers = {"Authorization": "KakaoAK 6341d1e76477fbac498a469adc87108d"}
result1 = json.loads(str(requests.get(url1, headers=headers).text))
result1['documents'][0]['address']['region_3depth_h_name']

'명일1동'

In [9]:
dong_list=[]

for i in range(len(cfdata.ADDR)):
    
    #카카오 API 주소검색
    
    url1 = 'https://dapi.kakao.com/v2/local/search/address.json?query='+ cfdata.ADDR[i]
    headers = {"Authorization": "KakaoAK 6341d1e76477fbac498a469adc87108d"}
    result1 = json.loads(str(requests.get(url1, headers=headers).text))
    
    if len(result1['documents']) == 0:    #주소검색이 되지 않는 주소일 시 건너뜀
        i += 1
        dong_list.append('0')
        
    else:
        addinfo = result1['documents'][0]['address']
        
        dong = addinfo['region_3depth_h_name']
        
        dong_list.append(dong)
        
print(len(dong_list))

24541


In [10]:
dong_list.count('0')

284

In [164]:
cfdata['region_2'] = dong_list

In [165]:
if cfdata['TRDSTATEGBN'][0] == 3 :
    print('tt')

tt


In [166]:
cfdata['DCBYMD'][0] < dt.datetime(2018,5,1)

True

In [167]:
cfdata['region_2'][0]

'성내3동'

In [168]:
tt = str(cfdata['DCBYMD'][5].year) + '.' + str(cfdata['DCBYMD'][5].month // 4 + 1 ) + '/4' + cfdata['region_2'][5]
tt

'2017.3/4천호2동'

In [171]:
type(fldata['기간'][0])

str

In [201]:
check_list = []

for i in range(len(fldata)):
    
    cf = fldata['기간'][i] + fldata['자치구'][i] + fldata['행정동'][i]
        
    check_list.append(cf)

len(check_list)

6363

In [202]:
check_list[:10]

['2017.1/4종로구사직동',
 '2017.1/4종로구삼청동',
 '2017.1/4종로구부암동',
 '2017.1/4종로구평창동',
 '2017.1/4종로구무악동',
 '2017.1/4종로구교남동',
 '2017.1/4종로구가회동',
 '2017.1/4종로구종로1.2.3.4가동',
 '2017.1/4종로구종로5·6가동',
 '2017.1/4종로구이화동']

In [203]:
fldata['check'] = check_list

In [204]:
fldata.iloc[4,3:25]

계          8407
0~4세        341
5~9세        416
10~14세      468
15~19세      541
20~24세      510
25~29세      431
30~34세      448
35~39세      573
40~44세      766
45~49세      862
50~54세      671
55~59세      666
60~64세      494
65~69세      371
70~74세      299
75~79세      283
80~84세      154
85~89세       70
90~94세       18
95~99세       12
100세 이상      13
Name: 4, dtype: object

In [176]:
cfdata['계'] = 0
cfdata['0~4세'] = 0
cfdata['5~9세'] = 0
cfdata['10~14세'] = 0
cfdata['15~19세'] = 0
cfdata['20~24세'] = 0
cfdata['25~29세'] = 0
cfdata['30~34세'] = 0
cfdata['35~39세'] = 0
cfdata['40~44세'] = 0
cfdata['45~49세'] = 0
cfdata['50~54세'] = 0
cfdata['55~59세'] = 0
cfdata['60~64세'] = 0
cfdata['65~69세'] = 0
cfdata['70~74세'] = 0
cfdata['75~79세'] = 0
cfdata['80~84세'] = 0
cfdata['85~89세'] = 0
cfdata['90~94세'] = 0
cfdata['95~99세'] = 0
cfdata['100세 이상'] = 0

In [200]:
fldata[fldata['check'] == c]

,기간,자치구,행정동,계,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,...,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세 이상,check
752,2017.2/4,관악구,신사동,26457,789.0,630.0,671.0,1029.0,1922.0,3085.0,...,1539.0,1128.0,868.0,565.0,299.0,132.0,40.0,17.0,9.0,2017.2/4신사동
781,2017.2/4,강남구,신사동,18259,559.0,679.0,867.0,1044.0,1202.0,1438.0,...,974.0,823.0,759.0,537.0,310.0,146.0,62.0,17.0,5.0,2017.2/4신사동


In [181]:
cfdata[cfdata.columns[17]]

0        0
1        0
2        0
3        0
4        0
        ..
24536    0
24537    0
24538    0
24539    0
24540    0
Name: 계, Length: 24541, dtype: int64

In [207]:
a=0
b=0

for i in range(len(cfdata)):
    #폐업한 가게
    if cfdata['TRDSTATEGBN'][i] == 3 :
        
        #연도+ . +분기+ /4 +행정동
        c = str(cfdata['DCBYMD'][i].year) + '.' + str(cfdata['DCBYMD'][i].month // 4 + 1 ) + '/4' + cfdata['region'][i] + cfdata['region_2'][i]
        
        for k in range(len(fldata)):
            if fldata['check'][k] == c :

                #3~24(계~100세이상) 3+j
                for j in range(20):
                    a = 17 + j
                    b= 3 + j

                    cfdata[cfdata.columns[a]][i] = fldata[fldata['check'] == c][fldata.columns[b]].item()

            
    #영업중인 가게
    else : 
        
        #가장 최신 인구(2020.3/4)
        c = '2020' + '.' + '3/4' + cfdata['region'][i] + cfdata['region_2'][i]
        for k in range(len(fldata)):
            if fldata['check'][k] == c :

                #3~24(계~100세이상) 행 추가
                for j in range(20):
                    a = 17 + j
                    b= 3 + j

                    cfdata[cfdata.columns[a]][i] = fldata[fldata['check'] == c][fldata.columns[b]].item()

len(cfdata)

<ipython-input-207-3290d701cdf2>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cfdata[cfdata.columns[a]][i] = fldata[fldata['check'] == c][fldata.columns[b]].item()
<ipython-input-207-3290d701cdf2>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cfdata[cfdata.columns[a]][i] = fldata[fldata['check'] == c][fldata.columns[b]].item()


24541

In [208]:
cfdata.iloc[:,17:]

,계,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,30~34세,35~39세,40~44세,...,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세 이상
0,24517,924,786,916,1289,1709,1883,1980,2252,1877,...,2081,1630,1136,852,560,276,129,46,14,10
1,25687,869,624,633,949,1540,2079,2417,2261,1825,...,2528,2090,1426,1052,694,374,162,74,17,13
2,22718,822,929,1195,1588,1573,1474,1496,1695,1803,...,1929,1593,1045,674,412,282,156,80,13,7
3,24336,945,809,887,1232,1718,1863,2029,2296,1889,...,2041,1603,1126,820,543,266,122,42,15,9
4,28628,1172,1140,1182,1655,2024,2099,2162,2400,2001,...,2613,2085,1325,838,552,349,201,72,15,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24536,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24537,45250,1564,2721,2396,1888,2348,3189,3186,3848,4050,...,3107,3120,2509,1843,1147,652,363,177,0,0
24538,20017,763,1051,1142,1122,1137,1226,1226,1586,1815,...,1326,1280,1044,763,457,258,168,76,0,0
24539,45250,1564,2721,2396,1888,2348,3189,3186,3848,4050,...,3107,3120,2509,1843,1147,652,363,177,0,0


In [209]:
cfdata.to_csv('./data/cfdata.csv',encoding='utf-8')